In [1]:
%run Functions.ipynb

In [4]:
def Data(assets,exchanger="binance", merge_with_spot=False, merge_with_liquidation=False,
         merge_with_bitfinex=False, merge_with_indexes=False ,last_date = "2020-08-07 00:00:00"):
    # If you want to update the data, run download funding data and download bitmex prices
    dfs = []
    for asset in assets:

        rates_future = pd.read_csv("/Users/paolo/Desktop/Thesis_code/data/%s-%s-funding-data.csv"% (asset,exchanger))
        rates_future["timestamp"] = [rates_future["timestamp"][i][:-4] if (len(str(rates_future["timestamp"][i]))>20) else rates_future["timestamp"][i] for i in range(len(rates_future))] # Some columns have also microseconds... need to format as gold format...
        rates_future = rates_future.set_index("timestamp", drop=True)

        prices_future = pd.read_csv("/Users/paolo/Desktop/Thesis_code/data/%s-1m-%s-data-future.csv" % (asset,exchanger))
        prices_future = prices_future.set_index("timestamp") # maybe not needed
        prices_future.index = pd.to_datetime(prices_future.index,) #maybe not needed
        if exchanger=="binance":
            prices_future.index = prices_future.index + timedelta(seconds= 60)  # Binance had candle time == open time. We need candle time == close time.

        df   = pd.merge(prices_future, rates_future[["fundingRate"]],right_index = True, left_index=True,how="left")
        df.index = df.index.tz_localize(None)

        if merge_with_liquidation:
            if(asset == "XBTUSD"): liquidations_bitmex = liquidation_data_xbt
            if(asset == "ETHUSD"): liquidations_bitmex = liquidation_data_eth
            if(asset == "XRPUSD"): liquidations_bitmex = liquidation_data_xrp
            df = pd.merge(df, liquidations_bitmex ,right_index = True, left_index=True,how="left")
        
        if merge_with_spot:
            binance_spot = pd.read_csv("/Users/paolo/Desktop/Thesis_code/data/%s-1m-binance-data-spot.csv" % asset)
            binance_spot = binance_spot.set_index(binance_spot.timestamp)
            binance_spot.index = pd.to_datetime(binance_spot.index,) #maybe not needed
            binance_spot.index = binance_spot.index + timedelta(seconds= 60)
            binance_spot.drop(["timestamp"], axis=1)   
            binance_spot["close_ref_binance"] = binance_spot["close"]

            df = pd.merge(df, binance_spot[["close_ref_binance"]],right_index = True, left_index=True,how="left")
            df["premium"] = df["close"] / df["close_ref_binance"] -1  #future/spot
        
        if merge_with_bitfinex:
            bitfinex_spot = pd.read_csv("/Users/paolo/Desktop/Thesis_code/data/XBTUSD-1m-bitifinex-data-spot.csv")
            bitfinex_spot.time = pd.to_datetime(bitfinex_spot.time, unit="s")
            bitfinex_spot["timestamp"] = bitfinex_spot.time
            bitfinex_spot.index = bitfinex_spot.timestamp
            bitfinex_spot["close_ref_bitfinex"] = bitfinex_spot["close"]
            
            df = pd.merge(df, bitfinex_spot[["close_ref_bitfinex"]],right_index = True, left_index=True,how="left")
            df["premium_btf"] = df["close"] / df["close_ref_bitfinex"] -1  #future/spot
        
        if merge_with_indexes:
            all_indexes = pd.read_csv("/Users/paolo/Desktop/Thesis_code/data/all_indexes.csv")
            all_indexes = all_indexes.set_index(all_indexes.timestamp, drop=True)
            all_indexes.index = pd.to_datetime(all_indexes.index,)
            all_indexes.index = all_indexes.index.tz_localize(None)
            all_indexes = all_indexes.drop(axis=1, labels="timestamp")
            df = pd.merge(df, all_indexes,right_index = True, left_index=True,how="left")
            
        df["random_signal"] = np.round(np.random.normal(0,1,len(df)),1)
        df = df.apply(pd.to_numeric, errors='coerce')
        df["fundingRate"][np.isnan(df["fundingRate"])] = 0
        df = df.dropna(subset=["open"])
        df["log_ret"] = np.log(df["close"] / df["open"])
        df["asset"] = asset
        df["exchanger"] = exchanger

        df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
        df = df[df.columns.drop(list(df.filter(regex='.1')))]

        windows = [5,60,1440] #[5,30,60,120,240,360,60*12,60*18,60*24]
        skips =   [5,60,1440] #[5,10,30,60,120,240,360,60*12,60*18,60*24] #skips   = [int(x / 5) for x in windows]
        df = window_return(df, windows, skips, asset)
        df = df[df.index<last_date]
        df = df[~df.index.duplicated(keep='first')]
        #df = cumulative_step(df, variables=["fundingRate"], windows = [3,6], freq=8*60 )
        dfs.append(df)
    return dfs

In [5]:
assets = ["ETHUSD", "XRPUSD"] #["BCHUSD","EOSUSD","LTCUSD","TRXUSD","ETCUSD","LINKUSD","ADAUSD","XLMUSD"] #(remove the t from usdt)
exchanger = "bitmex"
dfs =Data(assets, exchanger=exchanger,merge_with_spot=True, merge_with_liquidation=True)

for i in range(len(assets)):
    Save_pickle("/Users/paolo/Desktop/Thesis_code/pickles/df_%s_%s"%(assets[i],exchanger), dfs[i])

ETHUSD 5 5
ETHUSD 60 5
ETHUSD 1440 5
ETHUSD 60 60
ETHUSD 1440 60
ETHUSD 1440 1440
XRPUSD 5 5
XRPUSD 60 5
XRPUSD 1440 5
XRPUSD 60 60
XRPUSD 1440 60
XRPUSD 1440 1440


In [7]:
assets = ["XBTUSD","ETHUSD","XRPUSD","BCHUSD","EOSUSD","LTCUSD","TRXUSD","ETCUSD","LINKUSD","ADAUSD","XLMUSD","ALGOUSD"] #(remove the t from usdt)
exchanger = "binance"
dfs =Data(assets, exchanger=exchanger,merge_with_spot=False, merge_with_liquidation=False)

for i in range(len(assets)):
    Save_pickle("/Users/paolo/Desktop/Thesis_code/pickles/df_%s_%s"%(assets[i],exchanger), dfs[i])

XBTUSD 5 5
XBTUSD 60 5
XBTUSD 1440 5
XBTUSD 60 60
XBTUSD 1440 60
XBTUSD 1440 1440
ETHUSD 5 5
ETHUSD 60 5
ETHUSD 1440 5
ETHUSD 60 60
ETHUSD 1440 60
ETHUSD 1440 1440
XRPUSD 5 5
XRPUSD 60 5
XRPUSD 1440 5
XRPUSD 60 60
XRPUSD 1440 60
XRPUSD 1440 1440
BCHUSD 5 5
BCHUSD 60 5
BCHUSD 1440 5
BCHUSD 60 60
BCHUSD 1440 60
BCHUSD 1440 1440
EOSUSD 5 5
EOSUSD 60 5
EOSUSD 1440 5
EOSUSD 60 60
EOSUSD 1440 60
EOSUSD 1440 1440
LTCUSD 5 5
LTCUSD 60 5
LTCUSD 1440 5
LTCUSD 60 60
LTCUSD 1440 60
LTCUSD 1440 1440
TRXUSD 5 5
TRXUSD 60 5
TRXUSD 1440 5
TRXUSD 60 60
TRXUSD 1440 60
TRXUSD 1440 1440
ETCUSD 5 5
ETCUSD 60 5
ETCUSD 1440 5
ETCUSD 60 60
ETCUSD 1440 60
ETCUSD 1440 1440
LINKUSD 5 5
LINKUSD 60 5
LINKUSD 1440 5
LINKUSD 60 60
LINKUSD 1440 60
LINKUSD 1440 1440
ADAUSD 5 5
ADAUSD 60 5
ADAUSD 1440 5
ADAUSD 60 60
ADAUSD 1440 60
ADAUSD 1440 1440
XLMUSD 5 5
XLMUSD 60 5
XLMUSD 1440 5
XLMUSD 60 60
XLMUSD 1440 60
XLMUSD 1440 1440
ALGOUSD 5 5
ALGOUSD 60 5
ALGOUSD 1440 5
ALGOUSD 60 60
ALGOUSD 1440 60
ALGOUSD 1440 1440
